# 개요

# Prepare

# 1. 초기 세팅

## Library & data load

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
# 처리용 df와 오리지널 버전 남겨놓음
df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202108.csv",encoding='euc_kr')
org_df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202108.csv",encoding='euc_kr')

# Pre-Processing

## Basic info

In [3]:
df.head()

,품목명,단위,등급,가격,산지,친환경구분,입력일
0,[복숭아]복숭아(황도),2.5kg,특(1등),5500,충북 음성군,일반,20210731
1,[복숭아]복숭아(황도),4.5kg,특(1등),15000,충북 영동군,일반,20210731
2,[복숭아]복숭아(황도),4.5kg,특(1등),21000,충북 영동군,일반,20210731
3,[복숭아]복숭아(황도),3kg,특(1등),17000,충북 영동군,일반,20210731
4,[복숭아]복숭아(황도),4.5kg,특(1등),16000,경북 의성군,일반,20210731


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537840 entries, 0 to 537839
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   품목명     537840 non-null  object
 1   단위      537840 non-null  object
 2   등급      537840 non-null  object
 3   가격      537840 non-null  int64 
 4   산지      536542 non-null  object
 5   친환경구분   537840 non-null  object
 6   입력일     537840 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 28.7+ MB


In [5]:
# 산지 제외하고는 결측치 없는 것으로 확인
df.isnull().sum()

품목명         0
단위          0
등급          0
가격          0
산지       1298
친환경구분       0
입력일         0
dtype: int64

- 결측치 처리

In [6]:
###
# 양이 많지 않으므로 결측치 제거
df = df.dropna(axis=0)

- 영문명으로 전환

In [7]:
df = df.rename(columns = {'품목명':'prd',
                          '단위':'scale','등급':'class',
                          '가격':'price',
                          '산지':'origin',
                          '친환경구분':'eco',
                          '입력일':'reg_date'})

## column 별 정보

### class

In [8]:
# 이름 모두 영어로 변경
# 9등과 없음 모두 9로 변경
df['class'].value_counts().index

Index(['특(1등)', '상(2등)', '9등(등외)', '보통(3등)', '없음', '5등', '4등', '6등', '7등',
       '8등'],
      dtype='object')

In [9]:
df111 = pd.DataFrame([['특(1등)', 1], ['상(2등)',2], ['보통(3등)',3], ['9등(등외)',9], ['없음',9], ['5등',5], ['4등',4], ['6등',6], ['7등',7],['8등',8]],columns=['class','new_class'])

In [10]:
df2 = df.merge(df111)
df2 = df2.drop(columns='class')

In [11]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[복숭아]복숭아(황도),2.5kg,5500,충북 음성군,일반,20210731,1
1,[복숭아]복숭아(황도),4.5kg,15000,충북 영동군,일반,20210731,1
2,[복숭아]복숭아(황도),4.5kg,21000,충북 영동군,일반,20210731,1
3,[복숭아]복숭아(황도),3kg,17000,충북 영동군,일반,20210731,1
4,[복숭아]복숭아(황도),4.5kg,16000,경북 의성군,일반,20210731,1
...,...,...,...,...,...,...,...
536537,[자두]자두(추희),5kg,17000,충북 충주시,일반,20210827,8
536538,[양파]양파(일반),15kg,10800,전남 무안군,일반,20210828,8
536539,[기타과채]기타과채(감자),20kg,27000,강원 홍천군,일반,20210830,8
536540,[양파]양파(일반),15kg,10500,전남 무안군,일반,20210830,8


### eco

In [12]:
# 영문명으로 전환
# 전환기 -> 유기농 재배로 전환중
df2.loc[df2["eco"]=="일반","eco"] = "normal"
df2.loc[df2["eco"]=="우수농산물","eco"] = "good_prd"
df2.loc[df2["eco"]=="무농약","eco"] = "ecofriendly"
df2.loc[df2["eco"]=="유기농","eco"] = "organic"
df2.loc[df2["eco"]=="품질인증","eco"] = "certified"
df2.loc[df2["eco"]=="전환기","eco"] = "ing"

In [13]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[복숭아]복숭아(황도),2.5kg,5500,충북 음성군,normal,20210731,1
1,[복숭아]복숭아(황도),4.5kg,15000,충북 영동군,normal,20210731,1
2,[복숭아]복숭아(황도),4.5kg,21000,충북 영동군,normal,20210731,1
3,[복숭아]복숭아(황도),3kg,17000,충북 영동군,normal,20210731,1
4,[복숭아]복숭아(황도),4.5kg,16000,경북 의성군,normal,20210731,1
...,...,...,...,...,...,...,...
536537,[자두]자두(추희),5kg,17000,충북 충주시,normal,20210827,8
536538,[양파]양파(일반),15kg,10800,전남 무안군,normal,20210828,8
536539,[기타과채]기타과채(감자),20kg,27000,강원 홍천군,normal,20210830,8
536540,[양파]양파(일반),15kg,10500,전남 무안군,normal,20210830,8


### reg_date

In [14]:
df2['reg_date'] = df2['reg_date'].astype('str')

In [15]:
df2['reg_date'] = pd.to_datetime(df2['reg_date'].astype('str'),format='%Y%m%d')

In [16]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[복숭아]복숭아(황도),2.5kg,5500,충북 음성군,normal,2021-07-31,1
1,[복숭아]복숭아(황도),4.5kg,15000,충북 영동군,normal,2021-07-31,1
2,[복숭아]복숭아(황도),4.5kg,21000,충북 영동군,normal,2021-07-31,1
3,[복숭아]복숭아(황도),3kg,17000,충북 영동군,normal,2021-07-31,1
4,[복숭아]복숭아(황도),4.5kg,16000,경북 의성군,normal,2021-07-31,1
...,...,...,...,...,...,...,...
536537,[자두]자두(추희),5kg,17000,충북 충주시,normal,2021-08-27,8
536538,[양파]양파(일반),15kg,10800,전남 무안군,normal,2021-08-28,8
536539,[기타과채]기타과채(감자),20kg,27000,강원 홍천군,normal,2021-08-30,8
536540,[양파]양파(일반),15kg,10500,전남 무안군,normal,2021-08-30,8


### price

In [17]:
# 계산을 위해 scale 값 미리 변경
# 현재 값들은 string이므로 float 형태로 변환할 필요가 있음
# 몇몇 이상치 처리도 진행
def scale_checker(scale):
  if scale.startswith('.')==True:
    return format(float('0'+scale[:-2]),'.2f')
  else:
    return format(float((scale[:-2])),'.2f')
df2['scale'] = df2['scale'].apply(scale_checker)
df2['scale'] = df2['scale'].astype('float')
# df2['scale'] = format(round(df2['scale'].apply(pd.to_numeric),2),'.2f')

In [18]:
df2['scale'].unique()

array([2.500e+00, 4.500e+00, 3.000e+00, 4.000e+00, 1.500e+00, 1.000e+01,
       5.000e+00, 2.000e+00, 5.000e-01, 2.000e-01, 9.500e+00, 8.000e+00,
       7.000e+00, 1.100e+01, 9.000e+00, 6.000e+00, 1.000e+00, 2.000e+01,
       1.800e+01, 4.000e-01, 8.500e+00, 6.500e+00, 1.200e+01, 9.200e+00,
       1.500e+01, 3.500e+00, 5.500e+00, 1.700e+01, 1.900e+01, 1.400e+01,
       1.600e+01, 1.300e+01, 2.400e+00, 3.200e+00, 2.800e+00, 3.600e+00,
       3.800e+00, 3.300e+00, 4.200e+00, 8.000e+01, 9.000e-02, 7.500e+00,
       1.000e-01, 2.001e+01, 2.100e+01, 1.001e+01, 1.150e+01, 2.010e+00,
       4.010e+00, 8.600e+00, 8.010e+00, 1.200e+00, 8.000e-01, 3.400e+00,
       4.600e+00, 3.900e+00, 3.000e-01, 7.800e+00, 6.000e-01, 2.500e-01,
       6.000e-02, 1.500e-01, 5.000e-02, 2.200e+00, 2.100e+00, 2.600e+00,
       1.800e+00, 1.420e+01, 1.350e+01, 5.200e+00, 7.000e-01, 1.450e+01,
       4.800e+00, 1.300e+00, 8.400e+00, 1.600e+00, 1.050e+01, 1.040e+01,
       9.600e+00, 2.700e+00, 9.800e+00, 1.720e+01, 

In [19]:
# 1원짜리 거래 발견, 너무 거래값이 작은 값들은 삭제
pd.options.display.float_format = '{:.5f}'.format
df2['price'].describe()

count     536542.00000
mean       20456.60556
std        46501.86621
min            3.00000
25%         8500.00000
50%        14500.00000
75%        24000.00000
max     25008500.00000
Name: price, dtype: float64

In [20]:
df2 = df2.drop(df2[df2['price']<500].index)

In [21]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[복숭아]복숭아(황도),2.50000,5500,충북 음성군,normal,2021-07-31,1
1,[복숭아]복숭아(황도),4.50000,15000,충북 영동군,normal,2021-07-31,1
2,[복숭아]복숭아(황도),4.50000,21000,충북 영동군,normal,2021-07-31,1
3,[복숭아]복숭아(황도),3.00000,17000,충북 영동군,normal,2021-07-31,1
4,[복숭아]복숭아(황도),4.50000,16000,경북 의성군,normal,2021-07-31,1
...,...,...,...,...,...,...,...
536537,[자두]자두(추희),5.00000,17000,충북 충주시,normal,2021-08-27,8
536538,[양파]양파(일반),15.00000,10800,전남 무안군,normal,2021-08-28,8
536539,[기타과채]기타과채(감자),20.00000,27000,강원 홍천군,normal,2021-08-30,8
536540,[양파]양파(일반),15.00000,10500,전남 무안군,normal,2021-08-30,8


In [22]:
# kg 당 가격 생성

df2['price_kg'] = round((df2['price']/df2['scale']),2)

### prd
- 품목명 정리
- 품목 종류 정리

In [23]:
# 대부분 [상품 종류]상품명 으로 정리되어 있는걸 알 수 있음
df2['prd'].value_counts()

[오이]백다다기          15036
[호박]애호박            8924
[깻잎]깻잎             7490
[복숭아]황도            5460
[가지]가지             5222
                  ...  
[포도]네오마스캇             1
[파]쪽파(단)              1
[속새]속새(일반)            1
[실파]실파(기타)            1
[기타새우수입]기타새우수입        1
Name: prd, Length: 1492, dtype: int64

In [24]:
# 정리를 위해 상품명 삭제
def prd_sort(words):
  p = re.compile('[ㄱ-힣]+')
  word = re.search(p,words)
  return word.group()

df2['prd'] = df2['prd'].apply(prd_sort)

In [25]:
# 기타 품목 삭제
df2 = df2.drop(df2[df2['prd']=='기타'].index)

In [26]:
# 국산,수입산,깐,기타 삭제
def prd_sort2(words):
  p = re.compile('[^국산수입깐기타]+')
  word = re.search(p,str(words))
  return word.group()

df2["prd"] = df2["prd"].apply(prd_sort2)


In [27]:
# 몇몇 품목들 정리가 필요
a = sorted(df2['prd'].unique())
for i in range(len(a)):
  if i%15 != 0:
    print(a[i],end=' ')
  else:
    print(a[i],end=' ')
    print()

가리비 
가물치 가오리 가오리채 가자미 가지 갈치 감 감귤 감자 갑오징어 갓 강낭콩 개조개 건고구마순 건고추 
게 게지 겨자 고 고구마 고구마순 고둥 고들빼 고등어 고추 고추잎 곤달비 곤드레나물 골뱅이 과실류 
과일 과일류 과채 과채류 굴 귤 그레이프푸룻 근대 근채류 깻잎 꼴뚜 꽁치 꽈리고추 낙지 논우렁이 
눈볼대 느 다래 단감 달래 닭새우 당귀 당근 대구 대구포 대추 대파 더덕 도다리 도라지 
동부콩 동자개 동죽 두류 두릅 두리안 듀리안 딸 땅콩 라임 람부탄 레몬 리치 마늘 만가닥 
맛 망고 망고스턴 망고스틴 매생이 머위대 메 메론 명태 명태곤니 명태알 명태포 모시조개 목이 무 
무순 무우 무화과 문어 문어바지락 미꾸라지 미나리 미더덕 민들레 민물새우 민어 바나나 바지락 박 밤 
방울토마토 방풍 방풍나물 배 배추 백새우 백합 뱀장어 버섯 버섯류 버찌 벗 병어 복숭아 부세 
부지갱이 부추 붉은고추 붕어 붕장어 브로커리 브로코리 블루베리 비 비름 비트 빈스 빨간양배추 삐틀이 사과 
삼 삼치 상추 새꼬막 새송이 새싹 새우 새우살 새조개 샐러리 생강 생고추 석류 선동오징어 선인장열매 
세러리 셀러리 소라 속새 숙주나물 숫꽃게 시금치 실파 쌈채류 쌈추 쑥갓 아귀 아로니아 아보카도 아보카드 
아스파라가스 아스파라거스 아욱 알 알배 양배추 양상추 양송이 양채 양채류 양파 어류 얼갈이 얼갈이배추 여주 
연근 연어 열무 염고등어 엽경채류 엽채 엽채류 오렌지 오만둥이 오이 오징어 오징어몸통 오징어채 옥 완두콩 
용과 우렁쉥이 우렁이 유채 임연 자두 자몽 재첩 적겨자 적근대 적새우 적채 전복 절단 절단낙지 
조 조개살 중하 쪽파 쭈꾸미 참나물 참다래 참당귀 참외 참조 채소 천도복숭아 청경채 청어 체리 
총각무 취나물 치커리 칠게 칼라후라워 칼리플라워 칼리후라워 케일 코라비 코코넛 콩 콩나물 키위 키조개 키조개날개 
토란 토란대 토마토 파 파세리 파인애플 파프리카 패류 팽이 팽이버섯 포도 포장바지락 표고 표고버섯 풋고추 
풋옥 피망 피조개 해파리 향신료 호박 호박잎 홍고추 홍어 

In [28]:
### 예시
# df2[(df2['prd']=='유채') & (df2['price']==4500)]['prd'].unique()
# df2[df2['prd']=='유채']['prd'].unique()

In [29]:
### 원래 단어 뭐였는지 검색
# org_df[org_df['품목명'].str.contains('느')]['품목명'].unique()

In [30]:
### 현재 이름 검색
# df2[df2['prd']=='느']

In [33]:
###
# JG

# 고 -> 고수로 변경
df2.loc[df2["prd"]=="고","prd"] = "고수"
# 과실류, 과일류, 과채, 채소 -> 과채류로 통합
df2.loc[df2["prd"]=="과실류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일야채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일과채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일","prd"] = "과채류"
df2.loc[df2["prd"]=="과채","prd"] = "과채류"
df2.loc[df2["prd"]=="채소","prd"] = "과채류"
# 고들빼 -> 고들빼기로 변경
df2.loc[df2["prd"]=="고들빼","prd"] = "고들빼기"
# 꼴뚜 -> 꼴두기로 변경
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
# 그레이프푸룻 -> 자몽으로 통합
df2.loc[df2["prd"]=="그레이프푸룻","prd"] = "자몽"

# 느 -> 느타리버섯으로 통합
df2.loc[df2["prd"]=="느","prd"] = "느타리버섯"

# 대구 관련 품목들 정리
df2.loc[df2["prd"]=="대구고니","prd"] = "대구"
df2.loc[df2["prd"]=="대구머리","prd"] = "대구"
df2.loc[df2["prd"]=="대구포","prd"] = "대구"
# 듀리안 -> 두리안으로 통합
df2.loc[df2["prd"]=="듀리안","prd"] = "두리안"
# 딸 -> 딸기로 변경
df2.loc[df2["prd"]=="딸","prd"] = "딸기"

# 만가닥 -> 만가닥버섯으로 변경
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
# 맛 -> 맛조개로 변경
df2.loc[df2["prd"]=="맛","prd"] = "맛조개"
# 망고스턴 -> 망고스틴으로 수정
df2.loc[df2["prd"]=="망고스턴","prd"] = "망고스틴"
# 메 -> 메기로 변경
df2.loc[df2["prd"]=="메","prd"] = "메기"
# 명태 관련 품목들 정리
df2.loc[df2["prd"]=="명태곤니","prd"] = "명태"
df2.loc[df2["prd"]=="명태알","prd"] = "명태"
df2.loc[df2["prd"]=="명태포","prd"] = "명태"
# 무우 -> 무로 통합
df2.loc[df2["prd"]=="무우","prd"] = "무"

# 방풍 -> 방풍나물로 통합
df2.loc[df2["prd"]=="방풍","prd"] = "방풍나물"
# 버섯류 -> 버섯으로 변경
df2.loc[df2["prd"]=="버섯","prd"] = "버섯"
# 벗,버찌 -> 체리로 통합
df2.loc[df2["prd"]=="벗","prd"] = "체리"
df2.loc[df2["prd"]=="버찌","prd"] = "체리"
df2.loc[df2["prd"]=="벗찌","prd"] = "체리"
# 브로커리, 브로코리 -> 브로콜리로 통합
df2.loc[df2["prd"]=="브로커리","prd"] = "브로콜리"
df2.loc[df2["prd"]=="브로코리","prd"] = "브로콜리"

# 삼 -> 수삼으로 변경
df2.loc[df2["prd"]=="삼","prd"] = "수삼"
# 새우살 -> 새우로 변경
df2.loc[df2["prd"]=="새우살","prd"] = "새우"
# 세러리, 셀러리 -> 샐러리로 통합
df2.loc[df2["prd"]=="세러리","prd"] = "샐러리"
df2.loc[df2["prd"]=="셀러리","prd"] = "샐러리"
# 생고추 -> 고추로 통합
df2.loc[df2["prd"]=="생고추","prd"] = "고추"

# 아보카드 -> 아보카도로 변경
df2.loc[df2["prd"]=="아보카드","prd"] = "아보카도"
# 아스파라가스 -> 아스파라거스로 변경
df2.loc[df2["prd"]=="아스파라가스","prd"] = "아스파라거스"
# 알 -> 알타리무로 변경
df2.loc[df2["prd"]=="알","prd"] = "알타리무"
# 알배 -> 알배기로 변경
df2.loc[df2["prd"]=="알배","prd"] = "알배기"
# 양채 -> 양채류로 통합
df2.loc[df2["prd"]=="양채","prd"] = "양채류"
# 얼갈이 -> 얼갈이배추로 통합
df2.loc[df2["prd"]=="얼갈이","prd"] = "얼갈이배추"
# 엽채류, 엽채 -> 엽경채류로 통합
df2.loc[df2["prd"]=="엽채류","prd"] = "엽경채류"
df2.loc[df2["prd"]=="엽채","prd"] = "엽경채류"
# 오징어 관련 품목들 정리
df2.loc[df2["prd"]=="오징어다리","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어원양","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채","prd"] = "오징어"
df2.loc[df2["prd"]=="활복오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채원양","prd"] = "오징어"
df2.loc[df2["prd"]=="갑오징어살","prd"] = "오징어"
# 옥, 풋옥 -> 옥수수로 정리
df2.loc[df2["prd"]=="옥","prd"] = "옥수수"
df2.loc[df2["prd"]=="풋옥","prd"] = "옥수수"
# 임연 -> 임연수어로 정리
df2.loc[df2["prd"]=="임연","prd"] = "임연수어"


# 절단 -> 절단쭈꾸미 -> 쭈꾸미로 정리
df2.loc[df2["prd"]=="절단","prd"] = "쭈꾸미"
# 조개살 -> 조개로 정리
df2.loc[df2["prd"]=="조개살","prd"] = "조개"

# 참조 -> 참조기로 정리
df2.loc[df2["prd"]=="참조","prd"] = "참조기"

# 칼라후라워,칼리플라워, 칼리후라워 -> 컬리플라워로 통합
df2.loc[df2["prd"]=="칼라후라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리플라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리후라워","prd"] = "컬리플라워"
# 코라비 -> 콜라비로 통합
df2.loc[df2["prd"]=="코라비","prd"] = "콜라비"
# 키조개날개 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개날개","prd"] = "키조개"

# 팽이 -> 팽이버섯으로 통합
df2.loc[df2["prd"]=="팽이","prd"] = "팽이버섯"
# 파세리 -> 파슬리로 변경
df2.loc[df2["prd"]=="파세리","prd"] = "파슬리"
# 표고 -> 표고버섯으로 통합
df2.loc[df2["prd"]=="표고","prd"] = "표고버섯"
# 포장바지락 -> 바지락으로 변경
df2.loc[df2["prd"]=="포장바지락","prd"] = "바지락"

### 일단 여기까지 실행
# JY
#각굴 -> 굴로 통합
df2.loc[df2["prd"]=="각굴","prd"] = "굴"
#가오리채 -> 가오리로 통합
df2.loc[df2["prd"]=="가오리채","prd"] = "가오리"
# 게지 -> 키조개로 통합
df2.loc[df2["prd"]=="게지","prd"] = "키조개"
#고 -> 고구마로 통합
df2.loc[df2["prd"]=="고","prd"] = "고구마"
#꼴뚜 -> 꼴뚜기로 통합
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
#돗나물 -> 돌나물로 통합
df2.loc[df2["prd"]=="돗나물","prd"] = "돌나물"
#레드쉬 -> 레디쉬로 통합
df2.loc[df2["prd"]=="레드쉬","prd"] = "레디쉬"
#만가닥 -> 만가닥버섯으로 통합
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
#메 -> 메기로 통합
df2.loc[df2["prd"]=="메","prd"] = "메기"
#봄동 -> 봄동배추로 통합
df2.loc[df2["prd"]=="봄동","prd"] = "봄동배추"
#봉지굴 -> 굴로 통합
df2.loc[df2["prd"]=="봉지굴","prd"] = "굴"
#박 -> 수박으로 통합
df2.loc[df2["prd"]=="박","prd"] = "수박"
#백조 -> 조기로 통합 
df2.loc[df2["prd"]=="백조","prd"] = "조기"
#벅굴 -> 벚굴로 통합
df2.loc[df2["prd"]=="벅굴","prd"] = "벚굴"
#비 -> 비타민으로 통합
df2.loc[df2["prd"]=="비","prd"] = "비타민"
#빈스 -> 콩으로 통합
df2.loc[df2["prd"]=="빈스","prd"] = "콩"
#삐틀이 -> 고둥으로 통합
df2.loc[df2["prd"]=="삐틀이","prd"] = "고둥"
#새송이 -> 새송이버섯으로 통합
df2.loc[df2["prd"]=="새송이","prd"] = "새송이버섯"
#세트 -> 과채류로 통합
df2.loc[df2["prd"]=="세트","prd"] = "과채류"
#알 -> 알로애로 통합
df2.loc[df2["prd"]=="알","prd"] = "알로에"
#엄나무 -> 음나무로 통합
df2.loc[df2["prd"]=="엄나무","prd"] = "음나무"
#염고등어 -> 고등어로 통합
df2.loc[df2["prd"]=="염고등어","prd"] = "고등어"
#오만둥이 -> 미더덕으로 통합
df2.loc[df2["prd"]=="오만둥이","prd"] = "미더덕"
#오징어 관련 품목들 정리
df2.loc[df2["prd"]=="선동오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어몸통","prd"] = "오징어"
#우렁쉥이 -> 멍게로 통합
df2.loc[df2["prd"]=="우렁쉥이","prd"] = "멍게"
#우뭇가시리 -> 우뭇가사리로 통합
df2.loc[df2["prd"]=="우뭇가시리","prd"] = "우뭇가사리"
#적채 -> 적양배추로 통합
df2.loc[df2["prd"]=="적채","prd"] = "적양배추"
#절단낙지 -> 낙지로 통합
df2.loc[df2["prd"]=="절단낙지","prd"] = "낙지"
#조 -> 조개로 통합
df2.loc[df2["prd"]=="조","prd"] = "조개"
#청어원양 -> 청어로 통합
df2.loc[df2["prd"]=="청어원양","prd"] = "청어"
#키조개꼭지 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개꼭지","prd"] = "키조개"

# 암꽃게, 숫꽃게 -> 꽃게로 통합
df2.loc[df2["prd"]=="암꽃게","prd"] = "꽃게"
df2.loc[df2["prd"]=="숫꽃게","prd"] = "꽃게"
# 우렁 -> 우렁이로 통합
df2.loc[df2["prd"]=="우렁","prd"] = "우렁이"
# 알로애 -> 알로에로 변경
df2.loc[df2["prd"]=="알로애","prd"] = "알로에"
# 피마자잎 -> 피마자로 변경
df2.loc[df2["prd"]=="피미자잎","prd"] = "피마자"
# 식용허브 -> 허브로 변경
df2.loc[df2["prd"]=="식용허브","prd"] = "허브"
# 선인장열매 -> 선인장으로 변경
df2.loc[df2["prd"]=="선인장열매","prd"] = "선인장"
# 쌈추 -> 쌈채류로 변경
df2.loc[df2["prd"]=="쌈추","prd"] = "쌈채류"
# 조미제품 삭제
df2.drop(df2[df2['prd']=='조미제품'].index,inplace=True)
df2.drop(df2[df2['prd']=='어류원양'].index,inplace=True)

In [34]:
# 기존 products 리스트와 비교
ex_prd = pd.read_csv("../products.csv")
df2[~df2['prd'].isin(ex_prd['prd'])]['prd'].unique()

array(['토란대', '선인장'], dtype=object)

In [35]:
# 물품 리스트 최신화
new_prd = pd.DataFrame(df2['prd'].unique(),columns=['prd'])
# products = ex_prd.append(new_prd[~new_prd['prd'].isin(ex_prd['prd'])],ignore_index=True)
products = ex_prd.append(new_prd,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_2048/2619460125.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products = ex_prd.append(new_prd,ignore_index=True)


In [36]:
products.drop(columns=['Unnamed: 0'],inplace=True)
products.drop_duplicates(inplace=True)
products.reset_index(inplace=True,drop=True)

In [37]:
# 다음 작업을 위해 저장
products.to_csv('../products.csv')

In [38]:
# 전체 거래량과 품목량
# 비교를 위해 분포 파악
# 거래량이 너무 적은 품목은 삭제할 필요성이 있음

prd_lst = (df2['prd'].value_counts()).to_frame()
prd_lst.describe()

,prd
count,219.00000
mean,2447.44292
std,7793.83156
min,1.00000
25%,11.50000
50%,79.00000
75%,944.50000
max,87126.00000


In [39]:
# 50개 미만 품목은 비교가 힘드므로 삭제
# 줄어든 종류만 남기기

prd_lst = prd_lst[prd_lst['prd']>=50]
prd_lst.reset_index(inplace=True)
prd_lst = prd_lst.rename(columns = {'index':'prd','prd':'count'})

In [40]:
df3 = df2.merge(prd_lst)
df3[df3['count']>=50]['prd'].value_counts()

복숭아    87126
오이     31462
고추     31333
버섯     31036
호박     23716
       ...  
중하        51
키위        51
대추        51
백합        50
갈치        50
Name: prd, Length: 128, dtype: int64

In [41]:
# 줄어든 것 확인했으므로 count column은 삭제
df3 = df3.drop(columns='count')

### scale

In [42]:
np.set_printoptions(precision=2, suppress=True)
df3['scale'].unique()

array([  2.5 ,   4.5 ,   3.  ,   4.  ,   1.5 ,  10.  ,   5.  ,   2.  ,
        14.5 ,   1.8 ,   3.5 ,   1.  ,   6.  ,   8.  ,   5.5 ,   2.3 ,
        10.5 ,   9.  ,   7.  ,   1.2 ,   4.6 ,   0.5 ,   0.2 ,   0.3 ,
         9.5 ,  11.  ,   7.5 ,  13.  ,  12.  ,  10.8 ,   7.2 ,  10.7 ,
         8.5 ,   3.6 ,   3.7 ,   3.72, 125.  ,   2.2 ,  20.  ,  14.  ,
        18.  ,   4.8 ,   1.3 ,  15.  ,   0.4 ,   9.6 ,   7.7 ,  14.3 ,
         4.4 ,  17.  ,  15.4 ,   0.7 ,   3.3 ,   3.4 ,   4.7 ,  26.  ,
        21.  ,  30.  ,   6.5 ,   9.2 ,   3.8 ,   3.9 ,   0.1 ,   7.8 ,
         8.4 ,   2.7 ,   2.4 ,   9.8 ,   2.1 ,   7.6 ,   2.8 ,   2.25,
         5.7 ,   4.3 ,  12.3 ,   3.2 ,   2.9 ,   6.8 ,  11.3 ,   9.4 ,
         8.6 ,   7.4 ,   6.1 ,   6.4 ,   8.1 ,   5.9 ,   1.7 ,   2.6 ,
         6.3 ,   8.7 ,  11.8 ,   5.2 ,  12.5 ,   4.9 ,   6.7 ,   3.1 ,
         6.9 ,   5.8 ,   4.2 ,   6.2 ,  11.1 ,   5.3 ,   8.9 ,  10.2 ,
         8.8 ,   7.1 ,  11.5 ,   1.4 ,  19.  ,  16.  ,  23.  ,  25.  ,
      

In [47]:
# 50kg 이상 거래된 건들 이상치라 판단하고 확인 진행
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
97671,블루베리,125.00000,26000,미국,normal,2021-08-14,1,208.00000
259796,자두,54.00000,13000,전남 순천시,normal,2021-08-18,1,240.74000
405961,고구마순,200.00000,1300,충남 예산군,normal,2021-08-05,1,6.50000
405962,고구마순,200.00000,1350,충남 예산군,normal,2021-08-05,1,6.75000
405964,고구마순,200.00000,1200,충남 예산군,normal,2021-08-05,1,6.00000
405965,고구마순,200.00000,1250,충남 예산군,normal,2021-08-05,1,6.25000
448158,당근,50.00000,35000,부산 기장군,normal,2021-08-24,1,700.00000
448654,당근,50.00000,31500,부산 기장군,normal,2021-08-24,2,630.00000
448809,당근,50.00000,25500,부산 기장군,normal,2021-08-24,3,510.00000


In [48]:
# 스케일 오표기된 항목 수정

# 이상치 항목 삭제
df3 = df3.drop(df3[(df3['scale']>=50)].index)

In [49]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg


### origin

In [50]:
# 도,시, 국산, 수입 표기 분류 필요
df3['origin'].unique()

array(['충북 음성군', '충북 영동군', '경북 의성군', '충북 충주시', '전북 남원시', '전북 임실군',
       '경북 영천시', '경북 경산시', '대구 동구', '경북 영주시', '경기 이천시', '충북 충주시 가금면',
       '강원 원주시', '충북 괴산군', '경북 김천시', '경북 성주군', '경북 안동시', '전남 곡성군',
       '대구 북구', '강원 화천군', '전라북도 남원시', '경북 상주시', '경북', '경북 청도군', '경북 칠곡군',
       '경기 여주시', '충북 옥천군', '경북 예천군', '충북 제천시', '경남 창녕군', '충북',
       '경북 영주시 단산면', '경기 안성시', '경기 가평군 가평읍', '강원 춘천시', '경북 문경시',
       '청주시 청원구', '경북 영덕군', '충남 논산시', '강원 양구군', '경남 진주시', '서울시 강서구',
       '충북 청주시 상당구', '경기 가평군', '충남 서산시', '경기 여주군', '경북 영주시 순흥면', '강원 평창군',
       '강원 홍천군', '경북 문경시 가은읍', '강원 횡성군', '충북 청주시', '강원', '제주 서귀포시', '경남',
       '충남 당진시', '전북 무주군', '경북 군위군', '충남 천안시', '충남 부여군', '강원도 철원군',
       '경북 청송군', '경기 포천시', '서울 강동구', '경기 구리시', '서울 송파구', '경기 하남시',
       '경기 오산시', '강원 철원군', '경기 양평군', '경기 광주시', '경기 양주시', '경기 연천군',
       '경기 양주군', '경기도 고양시', '경기도 여주군', '경기 고양시', '경기 파주시', '경기 양평군 양동면',
       '경기', '경기 고양시일산구', '경기 고양시 덕양구', '경기 여주시 대신면', '경기도 파주시', '경기 평택시',
       '경기 남양주시', '경기 포천군'

In [51]:
# 해당 조건에 맞는 값 추출

splits = df3['origin'].str.split(' ')
cacul = splits.apply(lambda x: pd.Series(x))
df66 = pd.merge(df3,cacul, left_index=True, right_index=True, how='inner')
df66

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,0,1,2,3
0,복숭아,2.50000,5500,충북 음성군,normal,2021-07-31,1,2200.00000,충북,음성군,NaN,NaN
1,복숭아,4.50000,15000,충북 영동군,normal,2021-07-31,1,3333.33000,충북,영동군,NaN,NaN
2,복숭아,4.50000,21000,충북 영동군,normal,2021-07-31,1,4666.67000,충북,영동군,NaN,NaN
3,복숭아,3.00000,17000,충북 영동군,normal,2021-07-31,1,5666.67000,충북,영동군,NaN,NaN
4,복숭아,4.50000,16000,경북 의성군,normal,2021-07-31,1,3555.56000,경북,의성군,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
534755,갈치,10.00000,30000,국내산,normal,2021-08-30,3,3000.00000,국내산,NaN,NaN,NaN
534756,갈치,10.00000,310000,국내산,normal,2021-08-31,3,31000.00000,국내산,NaN,NaN,NaN
534757,갈치,10.00000,210100,국내산,normal,2021-08-31,3,21010.00000,국내산,NaN,NaN,NaN
534758,갈치,7.00000,50000,수입 세네갈,normal,2021-08-31,3,7142.86000,수입,세네갈,NaN,NaN


In [52]:
# 세부 카테고리 정리 (state, city 편집 시 여기서 시작)

df4 = df66.rename(columns = {0:'state',1:'city'})

# 2,3은 필요없는 정보이므로 삭제
df4 = df4.drop(columns=[2,3])

In [53]:
df4['state'].unique()

array(['충북', '경북', '전북', '대구', '경기', '강원', '전남', '전라북도', '경남', '청주시',
       '충남', '서울시', '제주', '강원도', '서울', '경기도', '광주', '수입산', '중국', '부산시',
       '뉴질랜드', '미국', '필리핀', '인천', '칠레', '성남시', '세종', '인천시', '광주시', '기타외국',
       '페루', '대만', '태국', '제주도', '경북/문경시', '부산', '경북/상주시', '경북/안동시', '호주',
       '오스트레일', '오스트레일리아', '남아프리카공화국', '대전', '전주시', '충북청주시상당구', '베트남',
       '콜롬비아', '인천광역시', '서울특별시', '멕시코', '가락동', '부산광역시', '제주특별자치도', '울산',
       '타이', '경기고양시일산구', '대구시', '브라질', '부천시', '과테말라', '에콰도르', '코스타리카',
       '에쿠아도르', '에콰돌', '인도네시아', '충청남도', '세종시', '남아프리카', '수입', '국내산'],
      dtype=object)

In [54]:
# 시,도 제거
def name_sort(words):
  p = re.compile('[^시도]+')
  word = re.search(p,str(words))
  return word.group()

df4["state"] = df4["state"].apply(name_sort)

# 오타도 제거

def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["state"] = df4["state"].apply(name_sort2)

In [55]:
df4["state"].unique()

array(['충북', '경북', '전북', '대구', '경기', '강원', '전남', '전라북', '경남', '청주', '충남',
       '서울', '제주', '광주', '수입산', '중국', '부산', '뉴질랜드', '미국', '필리핀', '인천',
       '칠레', '성남', '세종', '기타외국', '페루', '대만', '태국', '경북/문경', '경북/상주',
       '경북/안동', '호주', '오스트레일', '오스트레일리아', '남아프리카공화국', '대전', '전주', '충북청주',
       '베트남', '콜롬비아', '인천광역', '서울특별', '멕', '가락동', '부산광역', '제주특별자치', '울산',
       '타이', '경기고양', '브라질', '부천', '과테말라', '에콰', '코스타리카', '에쿠아', '에콰돌',
       '인', '충청남', '남아프리카', '수입', '국내산'], dtype=object)

In [56]:
# state
# 특이케이스 제거

# 국내
df4.loc[df4["state"]=="국내산","city"] = "미상"
df4.loc[df4["state"]=="국내산","state"] = "국산"
df4.loc[df4["state"]=="원양산","city"] = "미상"

## 도단위
df4.loc[df4["state"]=="충청북","state"] = "충북"
df4.loc[df4["state"]=="충청남","state"] = "충남"
df4.loc[df4["state"]=="전라북","state"] = "전북"
df4.loc[df4["state"]=="전라남","state"] = "전남"
df4.loc[df4["state"]=="경상북","state"] = "경북"
df4.loc[df4["state"]=="경상남","state"] = "경남"

## 시단위
### 광역시

df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="대구","city"] = "대구"
df4.loc[df4["state"]=="서울특별","city"] = "서울"
df4.loc[df4["state"]=="서울특별","state"] = "서울"
df4.loc[df4["state"]=="인천광역","city"] = "인천"
df4.loc[df4["state"]=="인천광역","state"] = "인천"
df4.loc[df4["state"]=="부산","city"] = "부산"
df4.loc[df4["state"]=="부산광역","city"] = "부산"
df4.loc[df4["state"]=="부산광역","state"] = "부산"
df4.loc[df4["state"]=="가락동","city"] = "서울"
df4.loc[df4["state"]=="가락동","state"] = "서울"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="제주","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","state"] = "제주"
df4.loc[df4["state"]=="제주특별자","city"] = "제주"
df4.loc[df4["state"]=="제주특별자","state"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","city"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","state"] = "제주"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="울산","city"] = "울산"
df4.loc[df4["state"]=="울산","state"] = "울산"

### 일반시
df4.loc[df4["state"]=="성남","city"] = "성남"
df4.loc[df4["state"]=="성남","state"] = "경기"
df4.loc[df4["state"]=="전주","city"] = "전주"
df4.loc[df4["state"]=="전주","state"] = "전북"
df4.loc[df4["state"]=="청주","city"] = "청주"
df4.loc[df4["state"]=="청주","state"] = "충북"
df4.loc[df4["state"]=="수원","city"] = "수원"
df4.loc[df4["state"]=="수원","state"] = "경기"
df4.loc[df4["state"]=="부천","city"] = "부천"
df4.loc[df4["state"]=="부천","state"] = "경기"
df4.loc[df4["state"]=="경기고양","city"] = "고양"
df4.loc[df4["state"]=="경기고양","state"] = "경기"
df4.loc[df4["state"]=="충북청주","city"] = "청주"
df4.loc[df4["state"]=="충북청주","state"] = "충북"
df4.loc[df4["state"]=="전북전주","city"] = "전주"
df4.loc[df4["state"]=="전북전주","state"] = "전북"
df4.loc[df4["state"]=="경북/문경","city"] = "문경"
df4.loc[df4["state"]=="경북/문경","state"] = "경북"
df4.loc[df4["state"]=="포항","city"] = "포항"
df4.loc[df4["state"]=="포항","state"] = "경북"
df4.loc[df4["state"]=="경북/안동","city"] = "안동"
df4.loc[df4["state"]=="경북/안동","state"] = "경북"
df4.loc[df4["state"]=="전남/무안군","city"] = "전남"
df4.loc[df4["state"]=="전남/무안군","state"] = "무안"
df4.loc[df4["state"]=="경북/구미","city"] = "구미"
df4.loc[df4["state"]=="경북/구미","state"] = "경북"
df4.loc[df4["state"]=="경북/상주","city"] = "상주"
df4.loc[df4["state"]=="경북/상주","state"] = "경북"
df4.loc[df4["state"]=="무안","state"] = "전남"


# 외국
## 아시아
df4.loc[df4["state"]=="칠레","city"] = "칠레"
df4.loc[df4["state"]=="칠레","state"] = "수입"
df4.loc[df4["state"]=="일본","city"] = "일본"
df4.loc[df4["state"]=="일본","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="우즈베크","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베크","state"] = "수입"
df4.loc[df4["state"]=="이스라엘","city"] = "이스라엘"
df4.loc[df4["state"]=="이스라엘","state"] = "수입"
df4.loc[df4["state"]=="터키","city"] = "터키"
df4.loc[df4["state"]=="터키","state"] = "수입"
df4.loc[df4["state"]=="중국","city"] = "중국"
df4.loc[df4["state"]=="중국","state"] = "수입"
df4.loc[df4["state"]=="타이","city"] = "태국"
df4.loc[df4["state"]=="타이","state"] = "수입"
df4.loc[df4["state"]=="태국","city"] = "태국"
df4.loc[df4["state"]=="태국","state"] = "수입"
df4.loc[df4["state"]=="필리핀","city"] = "필리핀"
df4.loc[df4["state"]=="필리핀","state"] = "수입"
df4.loc[df4["state"]=="우즈베키스탄","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베키스탄","state"] = "수입"
df4.loc[df4["state"]=="대만","city"] = "대만"
df4.loc[df4["state"]=="대만","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="이란","city"] = "이란"
df4.loc[df4["state"]=="이란","state"] = "수입"
df4.loc[df4["state"]=="러","city"] = "러시아"
df4.loc[df4["state"]=="러","state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"city"] = "인도네시아"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"city"] = "인도"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"state"] = "수입"

## 아메리카

df4.loc[df4["state"]=="캘리포니아","city"] = "미국"
df4.loc[df4["state"]=="캘리포니아","state"] = "수입"
df4.loc[df4["state"]=="포클랜드","city"] = "영국"
df4.loc[df4["state"]=="포클랜드","state"] = "수입"
df4.loc[df4["state"]=="콜롬비아","city"] = "콜롬비아"
df4.loc[df4["state"]=="콜롬비아","state"] = "수입"
df4.loc[df4["state"]=="에콰","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰","state"] = "수입"
df4.loc[df4["state"]=="에쿠아","city"] = "에콰도르"
df4.loc[df4["state"]=="에쿠아","state"] = "수입"
df4.loc[df4["state"]=="에콰돌","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰돌","state"] = "수입"
df4.loc[df4["state"]=="과테말라","city"] = "과테말라"
df4.loc[df4["state"]=="과테말라","state"] = "수입"
df4.loc[df4["state"]=="코스타리카","city"] = "코스타리카"
df4.loc[df4["state"]=="코스타리카","state"] = "수입"
df4.loc[df4["state"]=="멕","city"] = "멕시코"
df4.loc[df4["state"]=="멕","state"] = "수입"
df4.loc[df4["state"]=="페루","city"] = "페루"
df4.loc[df4["state"]=="페루","state"] = "수입"
df4.loc[df4["state"]=="미국","city"] = "미국"
df4.loc[df4["state"]=="미국","state"] = "수입"
df4.loc[df4["state"]=="아르헨티나","city"] = "아르헨티나"
df4.loc[df4["state"]=="아르헨티나","state"] = "수입"
df4.loc[df4["state"]=="브라질","city"] = "브라질"
df4.loc[df4["state"]=="브라질","state"] = "수입"

## 유럽
df4.loc[df4["state"]=="스페인","city"] = "스페인"
df4.loc[df4["state"]=="스페인","state"] = "수입"
df4.loc[df4["state"]=="우크라이나","city"] = "우크라이나"
df4.loc[df4["state"]=="우크라이나","state"] = "수입"
df4.loc[df4["state"]=="이탈리아","city"] = "이탈리아"
df4.loc[df4["state"]=="이탈리아","state"] = "수입"
df4.loc[df4["state"]=="불가리아","city"] = "불가리아"
df4.loc[df4["state"]=="불가리아","state"] = "수입"

## 아프리카
df4.loc[df4["state"]=="남아프리카","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카","state"] = "수입"
df4.loc[df4["state"]=="남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카공화국","state"] = "수입"
df4.loc[df4["state"]=="수입남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="수입남아프리카공화국","state"] = "수입"


## 오세아니아
df4.loc[df4["state"]=="뉴질랜드","city"] = "뉴질랜드"
df4.loc[df4["state"]=="뉴질랜드","state"] = "수입"
df4.loc[df4["state"]=="오스트레일","city"] = "호주"
df4.loc[df4["state"]=="오스트레일","state"] = "수입"
df4.loc[df4["state"]=="오스트레일리아","city"] = "호주"
df4.loc[df4["state"]=="오스트레일리아","state"] = "수입"
df4.loc[df4["state"]=="호주","city"] = "호주"
df4.loc[df4["state"]=="호주","state"] = "수입"
df4.loc[df4["state"]=="통가","city"] = "통가"
df4.loc[df4["state"]=="통가","state"] = "수입"
df4.loc[df4["state"]=="뉴칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴칼레","state"] = "수입"
df4.loc[df4["state"]=="뉴","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴","state"] = "수입"
df4.loc[df4["state"]=="누벨칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="누벨칼레","state"] = "수입"
# 기타
df4.loc[df4["state"]=="수입산","state"] = "수입"
df4.loc[df4["state"]=="기타외국","city"] = "미상"
df4.loc[df4["state"]=="기타외국","state"] = "수입"

In [57]:
df4[df4['state']=='nan']

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city


In [58]:
df4['state'].unique()

array(['충북', '경북', '전북', '대구', '경기', '강원', '전남', '경남', '충남', '서울', '제주',
       '광주', '수입', '부산', '인천', '세종', '대전', '울산', '국산'], dtype=object)

In [59]:
# city
# 우선 결측치 제거
df4.loc[df4["city"].isnull(),"city"] = "미상"

In [60]:
# 이후 시,군 등 제거
def name_sort(words):
  p = re.compile('[^시군]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort)

In [61]:
# \t 도 제거
def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort2)

In [62]:
df4['city'].unique()

array(['음성', '영동', '의성', '충주', '남원', '임실', '영천', '경산', '대구', '영주', '이천',
       '원주', '괴산', '김천', '성주', '안동', '곡성', '화천', '상주', '미상', '청도', '칠곡',
       '여주', '옥천', '예천', '제천', '창녕', '안성', '가평', '춘천', '문경', '청주', '영덕',
       '논산', '양구', '진주', '강서구', '서산', '평창', '홍천', '횡성', '제주', '당진', '무주',
       '위', '천안', '부여', '철원', '청송', '포천', '강동구', '구리', '송파구', '하남', '오산',
       '양평', '광주', '양주', '연천', '고양', '파주', '평택', '남양주', '김포', '양천구', '예산',
       '광명', '밀양', '하동', '나주', '금산', '무안', '함안', '태안', '남구', '인제', '순천',
       '영암', '용인', '중국', '부산', '뉴질랜드', '장흥', '화성', '진천', '고창', '서천', '영광',
       '미국', '함양', '담양', '청양', '공주', '아산', '필리핀', '증평', '고흥', '옹진', '칠레',
       '김해', '의령', '부천', '광양', '삼척', '성남', '강화', '홍성', '보령', '고성', '북구',
       '거창', '봉화', '포항', '보은', '장수', '단양', '정선', '익산', '부안', '강릉', '중랑구',
       '완주', '세종', '태백', '계양구', '영양', '계룡', '의정부', '영월', '흥', '합천', '안산',
       '익', '함평', '장성', '화순', '보성', '광산구', '해남', '서구', '거제', '청원', '정읍',
       '진안', '고령', '은평구', '동대문구', '도봉구', '

In [63]:
# 나머지 케이스 정리

# 국내
df4.loc[df4["city"]=="기타","city"] = "미상"
## 일반시
df4.loc[df4["city"]=="위","city"] = "군위"
df4.loc[df4["city"]=="고흥(나로도)","city"] = "고흥"
df4.loc[df4["city"]=="산","city"] = "군산"
df4.loc[df4["city"]=="장항(서천)","city"] = "서천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="순천(광양)","city"] = "순천"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="화성(평택)","city"] = "화성"
df4.loc[df4["city"]=="흥","city"] = "시흥"
df4.loc[df4["city"]=="목포(무안)","city"] = "목포"
df4.loc[df4["city"]=="목포(무안)","state"] = "전남"
df4.loc[df4["city"]=="보성(벌교)","city"] = "보성"
df4.loc[df4["city"]=="울진(후포)","city"] = "울진"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="순","city"] = "순창"
df4.loc[df4["city"]=="익","city"] = "익산"
df4.loc[df4["city"]=="미금","city"] = "성남"
df4.loc[df4["city"]=="주문진","city"] = "강릉"
df4.loc[df4["city"]=="전남","city"] = "무안"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="남"),"city"] = "남원"
df4.loc[(df4['prd']=="호박") & (df4['state']=='전북'),"city"] = "전주"
## 광역시
df4.loc[df4["city"]=="강서구","city"] = "서울"
df4.loc[df4["city"]=="은평구","city"] = "서울"
df4.loc[df4["city"]=="강동구","city"] = "서울"
df4.loc[df4["city"]=="마포구","city"] = "서울"
df4.loc[df4["city"]=="중구","city"] = "서울"
df4.loc[df4["city"]=="송파구","city"] = "서울"
df4.loc[df4["city"]=="도봉구","city"] = "서울"
df4.loc[df4["city"]=="중랑구","city"] = "서울"
df4.loc[df4["city"]=="양천구","city"] = "서울"
df4.loc[df4["city"]=="성동구","city"] = "서울"
df4.loc[df4["city"]=="강남구","city"] = "서울"
df4.loc[df4["city"]=="동대문구","city"] = "서울"
df4.loc[df4["city"]=="영등포구","city"] = "서울"
df4.loc[df4["city"]=="영등포","city"] = "서울"
df4.loc[df4["city"]=="서초구","city"] = "서울"
df4.loc[df4["city"]=="구로구","city"] = "서울"
df4.loc[df4["city"]=="동대문","city"] = "서울"
df4.loc[df4["city"]=="금천구","city"] = "서울"
df4.loc[df4["city"]=="동작구","city"] = "서울"
df4.loc[df4["city"]=="용산구","city"] = "서울"

df4.loc[df4["city"]=="광산구","city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="남구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="북구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="서구"),"city"] = "광주"
df4.loc[df4["city"]=="동구","city"] = "광주"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="남구"),"city"] = "인천"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="서구"),"city"] = "인천"
df4.loc[df4["city"]=="미추홀","city"] = "인천"
df4.loc[df4["city"]=="부평구","city"] = "인천"
df4.loc[df4["city"]=="남동구","city"] = "인천"
df4.loc[df4["city"]=="미추홀구","city"] = "인천"
df4.loc[df4["city"]=="계양구","city"] = "인천"
df4.loc[df4["city"]=="마산(고성,진해)","city"] = "창원"
df4.loc[(df4["state"]=="부산")&(df4["city"]=="북구"),"city"] = "부산"
df4.loc[df4["city"]=="오포","city"] = "부산"
df4.loc[df4["city"]=="부산진","city"] = "부산"
df4.loc[df4["city"]=="사상구","city"] = "부산"
df4.loc[df4["city"]=="부산진구","city"] = "부산"
df4.loc[df4["city"]=="북제주","city"] = "제주"
df4.loc[df4["city"]=="남제주","city"] = "제주"

# 해외
## 아시아
df4.loc[df4["city"]=="러","city"] = "러시아"
df4.loc[df4["city"]=="말레이지아","city"] = "말레이시아"
df4.loc[df4["city"]=="타이","city"] = "태국"
df4.loc[df4["city"]=="인도네","city"] = "인도네시아"
## 아메리카
df4.loc[df4["city"]=="멕","city"] = "멕시코"
## 아프리카
df4.loc[df4["city"]=="남아공산","city"] = "남아공"
## 유럽
## 오세아니아

In [64]:
# 기존 city 리스트와 비교
ex_city = pd.read_csv("../cities.csv")
df4[~df4['city'].isin(ex_city['city'])]['city'].unique()

array([], dtype=object)

In [65]:
# 리스트 최신화
new_city = pd.DataFrame(df4['city'].unique(),columns=['city'])
cities = ex_city.append(new_city,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_2048/1856950528.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = ex_city.append(new_city,ignore_index=True)


In [66]:
cities.drop(columns=['Unnamed: 0'],inplace=True)
cities.drop_duplicates(inplace=True)
cities.reset_index(inplace=True,drop=True)

In [67]:
cities.to_csv('../cities.csv')

In [68]:
# 처리 완료하였으니 df4는 백업으로 두고 df5 활용
df5 = df4.drop(columns='origin')

# 전처리 결과

# 파일 저장

In [69]:
df5.to_csv('../data_proceed/202108_proceed.csv')